# Oracle DB Access

## Autonomous DB, in the Cloud

For this Notebook, you need to have an Oracle Account (which is free).

And you need to have created a `RACES` User, as reqiured below.

- Get your Oracle Autonomous DataBase in the Cloud for free: <https://oracle.github.io/learning-library/data-management-library/autonomous-database/shared/adb-quickstart-workshop/freetier/>
- JDBC on ATP or ADW: <https://www.oracle.com/database/technologies/java-connectivity-to-atp.html>
- Oracle DB Code samples repo: <https://github.com/oracle/oracle-db-examples>

Log on to your account using <http://cloud.oracle.com>
Then go to your instance, and go to the `Tools` tab.
![Tools](./adw.png)

**Database Actions** will take your where you want to be.


### Using Maven to get to the JDBC driver

In [1]:
// Proxy settings
System.out.println(String.format("HTTP Proxy: %s", System.getProperty("HTTP_PROXY")));
System.out.println(String.format("HTTPS Proxy: %s", System.getProperty("HTTPS_PROXY")));

HTTP Proxy: null
HTTPS Proxy: null


**Driver version is important, make sure you get at least that one:**

In [2]:
// %maven com.oracle.database.jdbc:ojdbc10:19.12.0.0
%maven com.oracle.database.jdbc:ojdbc8-production:21.1.0.0

In [3]:
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.Driver;
import java.sql.PreparedStatement;
import java.sql.ResultSet;
import java.sql.ResultSetMetaData;
import java.sql.Statement;
import java.sql.SQLException;

import oracle.jdbc.OracleConnection;
import oracle.jdbc.pool.OracleDataSource;
import java.sql.DatabaseMetaData;

import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;
import java.util.Properties;
import java.util.stream.Collectors;

In [4]:
// For ATP and ADW - use the TNS Alias name along with the TNS_ADMIN when using 18.3 JDBC driver
final static String WALLET_PATH = "/Users/olivierlediouris/Wallet_OlivDBOne"; // This is on the machine this code is running on.
final static String TNS_NAME = "olivdbone_medium"; // From tnsname.ora, in the wallet folder.

// This user must have been created before running this code.
final static String DB_USER = "RACES";
final static String DB_PASSWORD = "AkeuCoucou_1";

// Insert statements
static String[] insertStmt = new String[] {
        "INSERT INTO TRACK VALUES (1, 'Auto Club Speedway', 'Fontana, CA', 92000, 1997)",
        "INSERT INTO TRACK VALUES (2, 'Chicagoland Speedway','Joliet, IL',75000,2001)",
        "INSERT INTO TRACK VALUES (3, 'Darlington Raceway','Darlington, SC',63000,1950)",
        "INSERT INTO TRACK VALUES (4, 'Daytona International Speedway','Daytona Beach, FL',168000,1959)",
        "INSERT INTO TRACK VALUES (5, 'Homestead-Miami Speedway','Homestead, FL',65000,1995)",
        "INSERT INTO TRACK VALUES (6, 'Kansas Speedway','Kansas City, KS',81687,2001)",
        "INSERT INTO TRACK VALUES (7, 'Martinsville Speedway','Ridgeway, VA',65000,1947)",
        "INSERT INTO TRACK VALUES (8, 'Michigan International Speedway','Brooklyn, MI',137243,1968)",
        "INSERT INTO TRACK VALUES (9, 'Phoenix International Raceway','Avondale, AZ',76812,1964)",

        "INSERT INTO  RACE VALUES (1, 'Rolex 24 At Daytona','DP/GT','January 26 January 27',1)",
        "INSERT INTO  RACE VALUES (2, 'Gainsco Grand Prix of Miami','DP/GT','March 29',2)",
        "INSERT INTO  RACE VALUES (3, 'Mexico City 250','DP/GT','April 19',2)",
        "INSERT INTO  RACE VALUES (4, 'Bosch Engineering 250 at VIR','GT','April 27',4)",
        "INSERT INTO  RACE VALUES (5, 'RumBum.com 250','DP/GT','May 17',5)",
        "INSERT INTO  RACE VALUES (6, 'Lime Rock GT Classic 250','GT','May 26',6)",
        "INSERT INTO  RACE VALUES (7, 'Sahlen''s Six Hours of the Glen','DP/GT','June 7',7)"
};

In [5]:
void executeStatement(String sql, Connection connection) throws SQLException {
    try (Statement statement = connection.createStatement()) {
        statement.executeUpdate(sql);
    } // Will close statement
}

In [6]:
void executeQuery(String sql, Connection connection) throws SQLException {
    executeQuery(sql, connection, false);
}
void executeQuery(String sql, Connection connection, boolean filtered) throws SQLException {
    try (Statement statement = connection.createStatement()) {
        try (ResultSet resultSet = statement.executeQuery(sql)) {

            ResultSetMetaData metaData = resultSet.getMetaData();
            int columnCount = metaData.getColumnCount();

            while (resultSet.next()) {
                List<Object> oneRow = new ArrayList<>();
                for (int i = 0; i < columnCount; i++) {
                    oneRow.add(resultSet.getObject(i + 1));
                }
                if (!filtered || (filtered && !((String)oneRow.get(0)).startsWith("BIN$"))) {
                    System.out.println(oneRow.stream().map(obj -> String.valueOf(obj)).collect(Collectors.joining(", ")));
                }
            }
        } // Will close result set
    } // Will close statement
}

In [7]:
Properties info = new Properties();
info.put(OracleConnection.CONNECTION_PROPERTY_USER_NAME, DB_USER);
info.put(OracleConnection.CONNECTION_PROPERTY_PASSWORD, DB_PASSWORD);
info.put(OracleConnection.CONNECTION_PROPERTY_DEFAULT_ROW_PREFETCH, "20");

String dbURL = String.format("jdbc:oracle:thin:@%s?TNS_ADMIN=%s", TNS_NAME, WALLET_PATH);
System.out.println(String.format("Connecting using %s ...", dbURL));

OracleDataSource ods = new OracleDataSource();
ods.setURL(dbURL);
ods.setConnectionProperties(info);


Connecting using jdbc:oracle:thin:@olivdbone_medium?TNS_ADMIN=/Users/olivierlediouris/Wallet_OlivDBOne ...


In [8]:
OracleConnection connection = (OracleConnection) ods.getConnection();

In [9]:
// Get the JDBC driver name and version
DatabaseMetaData dbmd = connection.getMetaData();
System.out.println("Driver Name: " + dbmd.getDriverName());
System.out.println("Driver Version: " + dbmd.getDriverVersion());
// Print some connection properties
System.out.println("Default Row Prefetch Value is: " + connection.getDefaultRowPrefetch());
System.out.println("Database Username is: " + connection.getUserName());

Driver Name: Oracle JDBC driver
Driver Version: 21.1.0.0.0
Default Row Prefetch Value is: 20
Database Username is: RACES


#### Part One, drop and re-create tables and constraints

In [10]:
try {
    executeStatement("DROP TABLE RACE", connection);
} catch (SQLException sqlEx) {
    sqlEx.printStackTrace();
}

In [11]:
try {
    executeStatement("DROP TABLE TRACK", connection);
} catch (SQLException sqlEx) {
    sqlEx.printStackTrace();
}

In [12]:
try {
    executeStatement("CREATE TABLE TRACK (\n" +
            "    TRACK_ID NUMBER,\n" +
            "    NAME VARCHAR2(48),\n" +
            "    LOCATION VARCHAR2(128),\n" +
            "    SEATING NUMBER,\n" +
            "    YEAR_OPENED NUMBER\n" +
            ")", connection);
} catch (SQLException sqlEx) {
    sqlEx.printStackTrace();
}

In [13]:
try {
    executeStatement("ALTER TABLE TRACK ADD CONSTRAINT TRACK_PK PRIMARY KEY(TRACK_ID)", connection);
} catch (SQLException sqlEx) {
    sqlEx.printStackTrace();
}

In [14]:
try {
    executeStatement("CREATE TABLE RACE (\n" +
            "    RACE_ID NUMBER,\n" +
            "    NAME VARCHAR2(48),\n" +
            "    RACE_CLASS VARCHAR2(12),\n" +
            "    RACE_DATE VARCHAR2(64),\n" +
            "    TRACK_ID NUMBER\n" +
            ")", connection);
} catch (SQLException sqlEx) {
    sqlEx.printStackTrace();
}

In [15]:
try {
    executeStatement("ALTER TABLE RACE ADD CONSTRAINT RACE_PK PRIMARY KEY(RACE_ID)", connection);
} catch (SQLException sqlEx) {
    sqlEx.printStackTrace();
}

In [16]:
try {
    executeStatement("ALTER TABLE RACE ADD CONSTRAINT RACE_FK_TRACK FOREIGN KEY (TRACK_ID) REFERENCES TRACK(TRACK_ID) ON DELETE CASCADE", connection);
} catch (SQLException sqlEx) {
    sqlEx.printStackTrace();
}

#### Part two, populate the Schema

In [17]:
connection.setAutoCommit(false); // Default is true

In [18]:
Arrays.stream(insertStmt).forEach(stmt -> {
    try {
        executeStatement(stmt, connection);
    } catch (SQLException sqlEx) {
        sqlEx.printStackTrace();
    }
});

In [19]:
// Commit ? If auto commit is off (on by default).
connection.commit();

#### Part three, some queries!

In [20]:
String sqlSelect;

In [21]:
sqlSelect = "SELECT * FROM TAB";
System.out.println("Tables:");
executeQuery(sqlSelect, connection, true); // Filter on BIN$*
System.out.println("----------------");

Tables:
PYTAB, TABLE, null
RACE, TABLE, null
TRACK, TABLE, null
----------------


In [22]:
sqlSelect = "SELECT track.name, " +
        "COUNT(race.race_id), " +
        "COUNT(*) FROM race " +
        "JOIN track ON track.Track_ID = race.Track_ID " +
        "GROUP BY track.name " +
        "HAVING COUNT(race.race_id) > 1";

System.out.println(String.format("A Query: %s", sqlSelect));
System.out.println("----------------");
executeQuery(sqlSelect, connection);
System.out.println("----------------");

A Query: SELECT track.name, COUNT(race.race_id), COUNT(*) FROM race JOIN track ON track.Track_ID = race.Track_ID GROUP BY track.name HAVING COUNT(race.race_id) > 1
----------------
Chicagoland Speedway, 2, 2
----------------


Done

In [23]:
if (connection != null) {
    connection.close();
}

Bye!

---